In [34]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt
import os

# Load data

In [37]:
with open('data/ActionNet/train_EMG_augmented.pkl', 'rb') as f:
    train_data = pd.read_pickle(f)
    
with open('data/ActionNet/test_EMG_augmented.pkl', 'rb') as f:
    test_data = pd.read_pickle(f)

# Add Labels column

In [27]:
classes ={'Slice a potato', 'Spread jelly on a bread slice', 'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Clear cutting board', 'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Spread almond butter on a bread slice', 'Clean a plate with a sponge', 'Slice a cucumber', 'Clean a pan with a sponge', 'Slice bread', 'Clean a plate with a towel', 'Pour water from a pitcher into a glass', 'Peel a cucumber', 'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Open/close a jar of almond butter', 'Peel a potato', 'Get/replace items from refrigerator/cabinets/drawers', 'Stack on table: 3 each large/small plates, bowls', 'Clean a pan with a towel'}
classes_map = {c: i for i, c in enumerate(classes)}
print(len(classes_map),"classes")
print(classes_map)

20 classes
{'Slice bread': 0, 'Clean a plate with a towel': 1, 'Pour water from a pitcher into a glass': 2, 'Slice a potato': 3, 'Spread jelly on a bread slice': 4, 'Peel a cucumber': 5, 'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 6, 'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 7, 'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 8, 'Clear cutting board': 9, 'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 10, 'Spread almond butter on a bread slice': 11, 'Clean a plate with a sponge': 12, 'Open/close a jar of almond butter': 13, 'Peel a potato': 14, 'Slice a cucumber': 15, 'Get/replace items from refrigerator/cabinets/drawers': 16, 'Stack on table: 3 each large/small plates, bowls': 17, 'Clean a pan with a towel': 18, 'Clean a pan with a sponge': 19}


In [56]:
def remove_synonyms(description):
    if description=="Get items from refrigerator/cabinets/drawers":
        return "Get/replace items from refrigerator/cabinets/drawers"
    elif description=="Open a jar of almond butter":
        return "Open/close a jar of almond butter"
    else:
        return description
    
def filter_dataset(data, name):
    filtered_dataset = data.copy()
    filtered_dataset['description'] = filtered_dataset['description'].apply(remove_synonyms)
    classes = filtered_dataset['description'].unique()
    print("Number of classes found: ", len(classes))
    print("Mapping:",classes_map)
    filtered_dataset['label'] = filtered_dataset['description'].apply(lambda x: classes_map[x])
    print(filtered_dataset.head(2))
    print("Saving filtered dataset...")
    pd.to_pickle(filtered_dataset, os.path.join(DATA_PATH, f'ActionNet_{name}_filtered.pkl'))
    

In [63]:
filter_dataset(s04_test, "test_S04")

Number of classes found:  7
Mapping: {'Slice bread': 0, 'Clean a plate with a towel': 1, 'Pour water from a pitcher into a glass': 2, 'Slice a potato': 3, 'Spread jelly on a bread slice': 4, 'Peel a cucumber': 5, 'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 6, 'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 7, 'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 8, 'Clear cutting board': 9, 'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 10, 'Spread almond butter on a bread slice': 11, 'Clean a plate with a sponge': 12, 'Open/close a jar of almond butter': 13, 'Peel a potato': 14, 'Slice a cucumber': 15, 'Get/replace items from refrigerator/cabinets/drawers': 16, 'Stack on table: 3 each large/small plates, bowls': 17, 'Clean a pan with a towel': 18, 'Clean a pan with a sponge': 19}
                          description  start   stop  

# Rectify

In [38]:
import os
import pandas as pd
import numpy as np

In [39]:
FS = 160  # Sampling frequency
CUTOFF = 5  # Cutoff frequency
NUM_CHANNELS=8

In [52]:
def rectify_signal(data):
    return np.abs(data)

def filter_signal(data, fs, cutoff, num_channels):
    for i in range(num_channels):
        data[:,i] = low_pass_filter(data[:,i], fs, cutoff)
    return data

def low_pass_filter(data, fs, cutoff, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, padlen=5)

def normalization(data, mean, std):
    """Normalize with mean and std
    data: (n_samples, n_channels)
    mean and std: (n_channels,)
    """
    return (data - mean) / std
    

def preprocessing(data, steps):
    for step in steps:
        data.apply(step)


In [53]:
print(train_data["myo_left_readings"].head())

40    [[4, 12, 21, -21, 3, -2, -5, 7], [-6, -5, -5, ...
40    [[-5, -7, -6, -66, -24, -23, -6, -14], [-5, -2...
40    [[-7, -12, -21, -38, 5, -10, -15, -22], [0, 2,...
40    [[2, -1, -3, -27, 0, -4, 0, -1], [-4, 5, 4, 5,...
40    [[1, -4, -4, 0, -2, -1, -1, 3], [1, 5, -3, 12,...
Name: myo_left_readings, dtype: object


In [54]:
# Rectify signal
rectified_data = train_data.copy()
rectified_data["myo_left_readings"] = rectified_data["myo_left_readings"].apply(rectify_signal)
print(rectified_data["myo_left_readings"].head())

40    [[4, 12, 21, 21, 3, 2, 5, 7], [6, 5, 5, 48, 3,...
40    [[5, 7, 6, 66, 24, 23, 6, 14], [5, 2, 6, 38, 8...
40    [[7, 12, 21, 38, 5, 10, 15, 22], [0, 2, 3, 10,...
40    [[2, 1, 3, 27, 0, 4, 0, 1], [4, 5, 4, 5, 7, 7,...
40    [[1, 4, 4, 0, 2, 1, 1, 3], [1, 5, 3, 12, 2, 1,...
Name: myo_left_readings, dtype: object


In [55]:
# Low pass filter
filtered_data = rectified_data.copy()
filtered_data["myo_left_readings"]= filtered_data["myo_left_readings"].apply(filter_signal, args=(FS, CUTOFF, NUM_CHANNELS))
print(filtered_data["myo_left_readings"].head())

40    [[6, 11, 22, 18, 2, 1, 5, 8], [6, 11, 20, 18, ...
40    [[4, 6, 3, 65, 25, 22, 5, 10], [3, 6, 5, 59, 2...
40    [[5, 12, 22, 35, 2, 9, 14, 21], [5, 11, 20, 32...
40    [[3, 1, 2, 26, 0, 3, 0, 0], [3, 2, 3, 24, 1, 4...
40    [[1, 4, 4, 1, 0, 1, 1, 2], [1, 3, 4, 2, 1, 1, ...
Name: myo_left_readings, dtype: object


In [56]:
# Normalization
normalized_data = filtered_data.copy()
normalized_data["myo_left_mean"] = normalized_data["myo_left_readings"].apply(lambda x: np.mean(x, axis=0))
normalized_data["myo_left_std"] = normalized_data["myo_left_readings"].apply(lambda x: np.std(x, axis=0))
myo_left_mean = np.mean(normalized_data["myo_left_mean"].to_list(), axis=0)
myo_left_std = np.mean(normalized_data["myo_left_std"].to_list(), axis=0)
print(myo_left_mean, myo_left_std)

[13.83767894 13.11577493 14.54656682 15.66527495 16.68707235 12.93012465
 10.64835843 10.58289213] [4.86291377 4.11740436 4.45829986 5.06286368 5.6143549  4.56827959
 3.66215016 3.88714868]


In [57]:
MEAN=[13.36730503, 12.64401049, 14.07886228, 15.19383317, 16.18250797, 12.46924118, 10.17692475, 10.10188793]
STD = [4.66570959, 3.92417575, 4.24558945, 4.84317504, 5.39038114, 4.37708144, 3.5060197, 3.72504386]

In [63]:
def preprocessing(data):
    sides=["left", "right"]
    data = data.copy()
    for side in sides:
        column = f"myo_{side}_readings"
        #print(data[column].head(1))
        data[column] = data[column].apply(rectify_signal)
        #print(data[column].head(1))
        data[column]= data[column].apply(filter_signal, args=(FS, CUTOFF, NUM_CHANNELS))
        #print(data[column].head(1))
        data[column] = data[column].apply(normalization, args=(MEAN, STD))
        #print(data[column].head(1))
    return data

In [65]:
processed=preprocessing(train_data)
pd.to_pickle(processed, os.path.join("data","ActionNet", "train_EMG_augmented.pkl"))